# はじめに

In [ ]:
//week16
//https://frostyfriday.org/blog/2022/09/30/week-16-intermediate/

//setup
use role sysadmin;
use database d_harato_db;
use schema ff;
create or replace file format json_ff_harato
    type = json
    strip_outer_array = TRUE;
    
create or replace stage week_16_frosty_stage
    url = 's3://frostyfridaychallenges/challenge_16/'
    file_format = json_ff_harato;

create or replace table week16 as
select t.$1:word::text word, t.$1:url::text url, t.$1:definition::variant definition  
from @week_16_frosty_stage (file_format => 'json_ff_harato', pattern=>'.*week16.*') t;

//確認
select * from week16 as w where word like 'l%' limit 5;

DEFINITIONの内容

```json
[
  {
    "license": {
      "name": "CC BY-SA 3.0",
      "url": "https://creativecommons.org/licenses/by-sa/3.0"
    },
    "meanings": [
      {
        "antonyms": [],
        "definitions": [
          {
            "antonyms": [],
            "definition": "That which results; the conclusion or end to which any course or condition of things leads, or which is obtained by any process or operation; consequence or effect.",
            "example": "the result of a course of action;  the result of a mathematical operation",
            "synonyms": []
          }
        ],
        "partOfSpeech": "noun",
        "synonyms": []
      }
    ],
    "phonetic": "/ɹɪˈzʌlt/",
    "phonetics": [
      {
        "audio": "https://api.dictionaryapi.dev/media/pronunciations/en/result-us.mp3",
        "license": {
          "name": "BY-SA 3.0",
          "url": "https://creativecommons.org/licenses/by-sa/3.0"
        },
        "sourceUrl": "https://commons.wikimedia.org/w/index.php?curid=1239703",
        "text": "/ɹɪˈzʌlt/"
      }
    ],
    "sourceUrls": [
      "https://en.wiktionary.org/wiki/result"
    ],
    "word": "result"
  }
]
```

# jsonファイルを構造化していく

In [ ]:
-- 一番外側のリストを外してみる
-- value列に入っているのがフラット化されたもの
select
    definition_flatten.*
from
    week16 as w16,
    lateral flatten(w16.definition) definition_flatten
limit
    5
;

出力結果のカラム説明
1. SEQ
入力レコードに関連付けられた一意のシーケンス番号です。

2. KEY
マップまたはオブジェクトの場合、この列には展開された値のキーが含まれます。

3. PATH
フラット化する必要があるデータ構造内の要素へのパスです。

4. INDEX
配列の場合は要素のインデックスで、それ以外の場合は NULLです。

5. VALUE
フラット化された配列/オブジェクトの要素の値です。

6. THIS
平坦化される要素（再帰的フラット化に有用）です。

```yaml
{
  "license": {
    "name": "CC BY-SA 3.0",
    "url": "https://creativecommons.org/licenses/by-sa/3.0"
  },
  "meanings": [
    {
      "antonyms": [],
      "definitions": [
        {
          "antonyms": [],
          "definition": "That which results; the conclusion or end to which any course or condition of things leads, or which is obtained by any process or operation; consequence or effect.",
          "example": "the result of a course of action;  the result of a mathematical operation",
          "synonyms": []
        }
      ],
      "partOfSpeech": "noun",
      "synonyms": []
    }
  ],
  "phonetic": "/ɹɪˈzʌlt/",
  "phonetics": [
    {
      "audio": "https://api.dictionaryapi.dev/media/pronunciations/en/result-us.mp3",
      "license": {
        "name": "BY-SA 3.0",
        "url": "https://creativecommons.org/licenses/by-sa/3.0"
      },
      "sourceUrl": "https://commons.wikimedia.org/w/index.php?curid=1239703",
      "text": "/ɹɪˈzʌlt/"
    }
  ],
  "sourceUrls": [
    "https://en.wiktionary.org/wiki/result"
  ],
  "word": "result"
}
```

In [ ]:
-- licenseの中身を取得したい
select
    definition_flatten.value:license as "ドット表記"
    , definition_flatten.value['license'] as "かっこ表記"
from
    week16 as w16,
    lateral flatten(w16.definition) definition_flatten
limit
    5
;

```json
{
    "name": "CC BY-SA 3.0",
    "url": "https://creativecommons.org/licenses/by-sa/3.0"
}
```

In [ ]:
-- nameの中身を取得したい
select
    definition_flatten.value:license:name as "ドット表記"
    , definition_flatten.value['license']['name'] as "かっこ表記"
from
    week16 as w16,
    lateral flatten(w16.definition) definition_flatten
limit
    5
;

*注意* \
要素名がSnowflakeSQL識別子ルールに準拠していない場合、名前を二重引用符で囲む必要がある。

```sql
-- This contains a blank.
SELECT src:"company name" FROM partners;

-- This does not start with a letter or underscore.
SELECT zipcode_info:"94987" FROM addresses;

-- This contains characters that are not letters, digits, or underscores, and
-- it does not start with a letter or underscore.
SELECT measurements:"#sPerSquareInch" FROM english_metrics;
```

In [ ]:
-- 各要素の中身を確認
select
    definition_flatten.value:license
    , definition_flatten.value:meanings
    , definition_flatten.value:phonetic
    , definition_flatten.value:phonetics
    , definition_flatten.value:sourceUrls
    , definition_flatten.value:word
from
    week16 as w16,
    lateral flatten(w16.definition) definition_flatten
limit
    5
;

-- 各々の結果をみてみると、meaningsとphoneticsとsourceUrlsで配列の存在を確認

```json
{
  "license": {
    "name": "CC BY-SA 3.0",
    "url": "https://creativecommons.org/licenses/by-sa/3.0"
  },
  "meanings": [
    {
      "antonyms": [],
      "definitions": [
        {
          "antonyms": [],
          "definition": "That which results; the conclusion or end to which any course or condition of things leads, or which is obtained by any process or operation; consequence or effect.",
          "example": "the result of a course of action;  the result of a mathematical operation",
          "synonyms": []
        }
    ],
  }
  "sourceUrls": [
    "https://en.wiktionary.org/wiki/result"
  ],
  "word": "result"
}
```

In [ ]:
-- 一旦meaningsに関して、リストを外す
select
    meanings_flatten.value
from
    week16 as w16,
    lateral flatten(w16.definition) definition_flatten,
    lateral flatten(definition_flatten.value:meanings) meanings_flatten
limit
    5
;

- meanings_flattenの内容
```json
{
  "antonyms": [],
  "definitions": [
    {
      "antonyms": [],
      "definition": "That which results; the conclusion or end to which any course or condition of things leads, or which is obtained by any process or operation; consequence or effect.",
      "example": "the result of a course of action;  the result of a mathematical operation",
      "synonyms": []
    }
  ],
  "partOfSpeech": "noun",
  "synonyms": []
}

```

In [ ]:
-- 同様に他2つに関しても実施
-- 一旦meaningsに関して、リストを外す
select
    meanings_flatten.value as meanings_flatten
    ,phonetics_flatten.value as phonetics_flatten
    , sourceUrls_flatten.value as sourceurls_flatten
from
    week16 as w16,
    lateral flatten(w16.definition) definition_flatten,
    lateral flatten(definition_flatten.value:meanings)   meanings_flatten,
    lateral flatten(definition_flatten.value:phonetics)  phonetics_flatten,
    lateral flatten(definition_flatten.value:sourceUrls) sourceUrls_flatten
limit
    5
;

- phonetics_flatten
```json
{
  "audio": "https://api.dictionaryapi.dev/media/pronunciations/en/result-us.mp3",
  "license": {
    "name": "BY-SA 3.0",
    "url": "https://creativecommons.org/licenses/by-sa/3.0"
  },
  "sourceUrl": "https://commons.wikimedia.org/w/index.php?curid=1239703",
  "text": "/ɹɪˈzʌlt/"
}
```

- sourceurls_flatten
```json
"https://en.wiktionary.org/wiki/result"
```

In [ ]:
-- 今回解答に必要な要素を書き出す
select
    word
    , url
    , meanings_flatten.value:antonyms as general_antonyms
    , meanings_flatten.value:synonyms as general_synonyms
    , meanings_definitions_flatten.value:definition as definition
    , meanings_definitions_flatten.value:example as example_if_applicable
    , meanings_definitions_flatten.value:antonyms as definitional_synonyms
    , meanings_definitions_flatten.value:synonyms as definitional_antonyms
from
    week16 as w16,
    lateral flatten(w16.definition) definition_flatten,
    lateral flatten(definition_flatten.value:meanings)  meanings_flatten,
    lateral flatten(meanings_flatten.value:definitions) meanings_definitions_flatten
limit
    5

In [ ]:
-- 必要なカラムを抽出してカウントを実施
with get_data as (
    select
        word
        , url
        , meanings_flatten.value:antonyms as general_antonyms
        , meanings_flatten.value:synonyms as general_synonyms
        , meanings_definitions_flatten.value:definition as definition
        , meanings_definitions_flatten.value:example as example_if_applicable
        , meanings_definitions_flatten.value:antonyms as definitional_synonyms
        , meanings_definitions_flatten.value:synonyms as definitional_antonyms
    from
        week16 as w16,
        lateral flatten(w16.definition) definition_flatten,
        lateral flatten(definition_flatten.value:meanings)   meanings_flatten,
        lateral flatten(meanings_flatten.value:definitions)   meanings_definitions_flatten
)

select count(word), count(distinct word) from get_data;

正解の数字 \
If correct, then
- count(word) should get you 32,295 rows
- count(distinct word) should get you 3,000 rows


ドキュメントを見ると下記の記載がある

```
OUTER => TRUE | FALSE
FALSE の場合、パスでアクセスできないか、フィールドまたはエントリがゼロであるために展開できない入力行は、出力から完全に省略されます。
TRUE の場合、ゼロ行展開用に正確に1行が生成されます（ KEY、 INDEX、 VALUE 列に NULL が含まれる）。

デフォルト： FALSE
```

In [ ]:
-- 必要なカラムを抽出
with get_data as (
    select
        word
        , url
        , meanings_flatten.value:antonyms as general_antonyms
        , meanings_flatten.value:synonyms as general_synonyms
        , meanings_definitions_flatten.value:definition as definition
        , meanings_definitions_flatten.value:example as example_if_applicable
        , meanings_definitions_flatten.value:antonyms as definitional_synonyms
        , meanings_definitions_flatten.value:synonyms as definitional_antonyms
    from
        week16 as w16,
        lateral flatten(w16.definition, outer => TRUE) definition_flatten,
        lateral flatten(definition_flatten.value:meanings, outer => TRUE)  meanings_flatten,
        lateral flatten(meanings_flatten.value:definitions, outer => TRUE) meanings_definitions_flatten
)

select count(word), count(distinct word) from get_data
;

In [ ]:
-- 必要なカラムを抽出
with get_data as (
    select
        word
        , url
        , definition_flatten.value as general_antonyms
        , definition_flatten_array.value as general_antonyms_array
        , definition_flatten_object.value as definition_flatten_object
    from
        week16 as w16,
        lateral flatten(w16.definition) definition_flatten,
        lateral flatten(w16.definition, outer => TRUE, mode => 'OBJECT') definition_flatten_object,
        lateral flatten(w16.definition, outer => TRUE, mode => 'ARRAY') definition_flatten_array
    where true
        and URL in ('https://api.dictionaryapi.dev/api/v2/entries/en/mm-hmm', 'https://api.dictionaryapi.dev/api/v2/entries/en/result')
)

select * from get_data

今回の出力カラムは下記の部分 → 配列の箇所
```json
  "meanings": [
    {
      "antonyms": [],
      "definitions": [
        {
          "antonyms": [],
          "definition": "That which results; the conclusion or end to which any course or condition of things leads, or which is obtained by any process or operation; consequence or effect.",
          "example": "the result of a course of action;  the result of a mathematical operation",
          "synonyms": []
        }
      ]
```

In [ ]:
-- 必要なカラムを抽出
with get_data as (
    select
        word
        , url
        , definition_flatten.value as general_antonyms
        , meanings_flatten.value:antonyms as general_antonyms
        , meanings_flatten.value:synonyms as general_synonyms
        , meanings_definitions_flatten.value:definition as definition
        , meanings_definitions_flatten.value:example as example_if_applicable
        , meanings_definitions_flatten.value:antonyms as definitional_synonyms
        , meanings_definitions_flatten.value:synonyms as definitional_antonyms
    from
        week16 as w16,
        lateral flatten(w16.definition, outer => TRUE, mode => 'ARRAY') definition_flatten,
        lateral flatten(definition_flatten.value:meanings, outer => TRUE, mode => 'ARRAY')   meanings_flatten,
        lateral flatten(meanings_flatten.value:definitions, outer => TRUE, mode => 'ARRAY')   meanings_definitions_flatten
)

select count(word), count(distinct word) from get_data
;